## Deliverable 2. Create a Customer Travel Destinations Map.

In [51]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [52]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,2.88,93,100,21.41,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,55.51,44,75,19.57,broken clouds
2,2,Pag,HR,44.4450,15.0575,45.18,66,30,9.42,scattered clouds
3,3,Ribeira Grande,PT,38.5167,-28.7000,61.20,63,20,6.91,few clouds
4,4,Mataura,NZ,-46.1927,168.8643,61.48,96,100,1.88,overcast clouds


In [53]:
# 2. Prompt the user to enter minimum and maximum temperature criteria (6.5.3)
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [64]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Port Moresby,PG,-9.4431,147.1797,76.95,82,100,5.59,heavy intensity rain
8,8,Avarua,CK,-21.2078,-159.7750,80.65,83,40,10.36,scattered clouds
10,10,Moron,AR,-34.6534,-58.6198,85.95,48,9,7.49,clear sky
18,18,Pemba,MZ,-12.9740,40.5178,76.14,88,100,7.70,overcast clouds
23,23,Bambous Virieux,MU,-20.3428,57.7575,80.85,78,40,8.05,scattered clouds
30,30,Butaritari,KI,3.0707,172.7902,81.75,77,12,19.33,light rain
34,34,Atuona,PF,-9.8000,-139.0333,77.23,80,29,14.54,light rain
35,35,Kapaa,US,22.0752,-159.3190,77.32,79,100,1.01,overcast clouds
37,37,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,78,40,12.66,thunderstorm
43,43,Georgetown,MY,5.4112,100.3354,83.89,87,20,2.30,few clouds


In [65]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                176
City                   176
Country                175
Lat                    176
Lng                    176
Max Temp               176
Humidity               176
Cloudiness             176
Wind Speed             176
Current Description    176
dtype: int64

In [66]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df=preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                175
City                   175
Country                175
Lat                    175
Lng                    175
Max Temp               175
Humidity               175
Cloudiness             175
Wind Speed             175
Current Description    175
dtype: int64

In [67]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Port Moresby,PG,76.95,heavy intensity rain,-9.4431,147.1797,
8,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
10,Moron,AR,85.95,clear sky,-34.6534,-58.6198,
18,Pemba,MZ,76.14,overcast clouds,-12.9740,40.5178,
23,Bambous Virieux,MU,80.85,scattered clouds,-20.3428,57.7575,
30,Butaritari,KI,81.75,light rain,3.0707,172.7902,
34,Atuona,PF,77.23,light rain,-9.8000,-139.0333,
35,Kapaa,US,77.32,overcast clouds,22.0752,-159.3190,
37,Arraial Do Cabo,BR,84.16,thunderstorm,-22.9661,-42.0278,
43,Georgetown,MY,83.89,few clouds,5.4112,100.3354,


In [68]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Port Moresby,PG,76.95,heavy intensity rain,-9.4431,147.1797,Lamana Hotel (Port Moresby)
8,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Moron,AR,85.95,clear sky,-34.6534,-58.6198,Hotel Morón
18,Pemba,MZ,76.14,overcast clouds,-12.9740,40.5178,Guesthouse Samotina
23,Bambous Virieux,MU,80.85,scattered clouds,-20.3428,57.7575,Casa Tia Villa


In [70]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))